In [48]:
def run_mlflow_workflow(sqlite_path, batch_size=64, number_epochs=10):
    import uuid
    import mlflow

    mlflow.set_tracking_uri(
        f"sqlite:///" f"{sqlite_path}"
    )
    experiment_name = "LinearRegression"
    experiment_id = mlflow.create_experiment(
        experiment_name + str(uuid.uuid4())
    )
    with mlflow.start_run(experiment_id=experiment_id) as run:
        mlflow.log_params({"number_epochs": number_epochs})
        mlflow.log_params({"batch_size": batch_size})

        print("\nTrained model")
        mlflow.log_metric("loss", 0.04)
        return run.info.run_uuid

In [4]:
from flowcept import MLFlowInterceptor
interceptor = MLFlowInterceptor()

In [5]:
from flowcept import FlowceptConsumerAPI
consumer = FlowceptConsumerAPI(interceptor)
consumer.start()

Flowcept Interceptor <class 'flowcept.flowceptor.plugins.mlflow.mlflow_interceptor.MLFlowInterceptor'> starting...
... ok!
Flowcept Consumer starting...
Watching /Users/rsr/Library/CloudStorage/OneDrive-OakRidgeNationalLaboratory/ORNL_Pendrive/dev/flowcept/mlflow.db
Ok, we're consuming messages!


In [6]:
mlflow_run_id = run_mlflow_workflow(interceptor.settings.file_path)
mlflow_run_id


Trained model


'7350e40e10be43c6948ebb1cdff275cd'

We need to intercept this Run: 7350e40e10be43c6948ebb1cdff275cd
Going to send to Redis an intercepted message:
	{"task_id": "7350e40e10be43c6948ebb1cdff275cd", "utc_timestamp": 1675796749.476072, "status": "FINISHED", "used": {"batch_size": "64", "number_epochs": "10"}, "generated": {"loss": 0.04}, "plugin_id": "mlflow", "user": "root", "experiment_id": "super-experiment", "sys_name": "Darwin", "node_name": "MAC132633", "login_name": "rsr", "public_ip": "2620:0:2b30:e0::4b9", "private_ip": "10.158.26.233"}
An intercepted message was received.
Time to flush!


In [7]:
from flowcept import TaskQueryAPI
query_api = TaskQueryAPI()

Ok, webserver is ready to receive requests.


In [8]:
_filter = {"task_id": mlflow_run_id}
query_api.query(_filter)

[{'task_id': '7350e40e10be43c6948ebb1cdff275cd',
  'utc_timestamp': 1675796749.476072,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True}]

In [19]:
def get_utc_minutes_ago(minutes_ago=1):
    from datetime import datetime, timedelta
    now = datetime.utcnow()
    rounded = now - timedelta(minutes=now.minute % minutes_ago + minutes_ago,
                              seconds=now.second,
                              microseconds=now.microsecond)
    return rounded.timestamp()

In [22]:
get_utc_minutes_ago(60)

1675792800.0

In [32]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment"
}
query_api.query(_filter)

[{'task_id': 'dd167381101d46b1a7ec43db36ba52eb',
  'utc_timestamp': 1675796172.583409,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True},
 {'task_id': '9faf2ef9125e4e8a8b33df7edc02879b',
  'utc_timestamp': 1675796220.932569,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True},
 {'task_id': '69dd04cbea944c5b80b723426079b187',
  'utc_timestamp': 1675796229.67979,
  'status': 'FINISHED',
  'u

An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!


In [35]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment",
    "plugin_id": {"$in": ['mlflow', 'dask']}
}
docs = query_api.query(_filter)
docs

[{'task_id': 'dd167381101d46b1a7ec43db36ba52eb',
  'utc_timestamp': 1675796172.583409,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True},
 {'task_id': '9faf2ef9125e4e8a8b33df7edc02879b',
  'utc_timestamp': 1675796220.932569,
  'status': 'FINISHED',
  'used': {'batch_size': '64', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True},
 {'task_id': '69dd04cbea944c5b80b723426079b187',
  'utc_timestamp': 1675796229.67979,
  'status': 'FINISHED',
  'u

In [44]:
_filter = {
    "utc_timestamp": { "$gte" : get_utc_minutes_ago(60) },
    "experiment_id": "super-experiment",
    "plugin_id": 'dask',
    "generated.batch_size": { "$gte" : 0 }
}
docs = query_api.query(filter=_filter)
batch_sizes = []
for doc in docs:
    print(f"task={doc['task_id']}, generated batch_size={doc['generated']['batch_size']}")
    batch_sizes.append(doc['generated']['batch_size'])
batch_sizes

task=dummy_func3-5711dd02e7574d2e484d545a9827112d, generated batch_size=19


[19]

In [49]:
for batch_size in batch_sizes:
    mlflow_task = run_mlflow_workflow(interceptor.settings.file_path, batch_size=batch_size)
    print(mlflow_task)


Trained model
7f515374639d4631b1fcaaf4c0365902
We need to intercept this Run: 7f515374639d4631b1fcaaf4c0365902
Going to send to Redis an intercepted message:
	{"task_id": "7f515374639d4631b1fcaaf4c0365902", "utc_timestamp": 1675799022.065446, "status": "FINISHED", "used": {"batch_size": "19", "number_epochs": "10"}, "generated": {"loss": 0.04}, "plugin_id": "mlflow", "user": "root", "experiment_id": "super-experiment", "sys_name": "Darwin", "node_name": "MAC132633", "login_name": "rsr", "public_ip": "2620:0:2b30:e0::4b9", "private_ip": "10.158.26.233"}
An intercepted message was received.
Time to flush!


In [50]:
_filter = {
    "task_id": mlflow_task
}
docs = query_api.query(filter=_filter)
docs

[{'task_id': '7f515374639d4631b1fcaaf4c0365902',
  'utc_timestamp': 1675799022.065446,
  'status': 'FINISHED',
  'used': {'batch_size': '19', 'number_epochs': '10'},
  'generated': {'loss': 0.04},
  'plugin_id': 'mlflow',
  'user': 'root',
  'experiment_id': 'super-experiment',
  'sys_name': 'Darwin',
  'node_name': 'MAC132633',
  'login_name': 'rsr',
  'public_ip': '2620:0:2b30:e0::4b9',
  'private_ip': '10.158.26.233',
  'debug': True}]

An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
Time to flush!
An intercepted message was received.
An intercepted message was received.
An intercepted message was received.
